개인 GPU 가 없는 사람을 위한, Google Colab 상에서 FastAPI 를 통해 딥러닝 Inference Server 를 띄우는 코드입니다.

- Last Update: 2024/02/03
- Written by `@dhdbsrlw`

## 1. Inference.py

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install torch diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.0 MB/s eta 0:00:00


In [3]:
# Install and import requirements
import torch
import diffusers
from PIL import Image

from torch import autocast
from diffusers import StableDiffusionPipeline


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

### 허깅페이스 허브로부터 Ckpt 를 받아와 1) 모델(SD) 로드 및 2) 이미지 생성 하는 함수입니다.

In [4]:

# ------------------------------------------------------

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols*w, i//cols*h))
    return grid


# ------------------------------------------------------

# Load the model
def load_model(model_id):
    # model_id = "sd-dreambooth-library/cat-toy"  # @param {type:"string"}
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id, torch_dtype=torch.float16).to("cuda")
    return pipe


# ------------------------------------------------------

# Run the Stable Diffusion pipeline on Colab
# Don't forget to use the `sks` token in your prompt

def generate(pipe, prompt):
    # prompt = "a photo of sks toy floating on a ramen bowl" #@param {type:"string"}

    num_samples = 1  # {type:"number"}
    num_rows = 1  # {type:"number"}

    all_images = []
    for _ in range(num_rows):
        images = pipe(prompt, num_images_per_prompt=num_samples,
                      num_inference_steps=50, guidance_scale=7.5).images
        all_images.extend(images)

    grid = image_grid(all_images, num_samples, num_rows)
    return grid


## 특정 프롬프트에 맞게 이미지 생성 테스트 (Optional)

In [12]:
pet_list = [ 'zero']
# model_list = ['dhdbsrlw/pet-onion', 'dhdbsrlw/pet-monga', 'dhdbsrlw/pet-zero']
prompt_list = ['running the park', 'training', 'sitting on the sofa', 'taking a break', 'playing with friends', 'eating dog food', 'wearing clothes', 'sleeping inside the fence']


In [13]:
from PIL import Image

for pet_name in pet_list:
  print('start loading...')
  pipe = load_model(model_id=f'dhdbsrlw/pet-{pet_name}')

  print('start generating...')
  for action in prompt_list:
    grid = generate(pipe, f'a photo of sks {pet_name} {action}')
    print(grid)

    print('saving an image...') # 생성한 이미지를 저장
    filename = f'{pet_name}_{action}.jpg'
    save_path = f'/content/gdrive/MyDrive/AI/ET/{filename}'
    grid.save(save_path)


start loading...


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

...

<PIL.Image.Image image mode=RGB size=512x512 at 0x7A9098848550>
saving an image...


  0%|          | 0/50 [00:00<?, ?it/s]

<PIL.Image.Image image mode=RGB size=512x512 at 0x7A9096313E50>
saving an image...


  0%|          | 0/50 [00:00<?, ?it/s]

<PIL.Image.Image image mode=RGB size=512x512 at 0x7A9096312BC0>
saving an image...


## 2. Main.py (실제 서버 구동 위해 필요한 FastAPI 코드 )

In [5]:
!pip install fastapi==0.103.2 nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.


In [6]:
!ngrok authtoken {TOKEN} # NEED TO PLUGIN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [15]:
# !uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [8079] using StatReload
INFO:     Started server process [8081]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [8081]
INFO:     Stopping reloader process [8079]


In [7]:
from PIL import Image

pet_name = 'onion'


In [10]:
from PIL import Image

print('start loading...')
pipe = load_model(model_id=f'dhdbsrlw/pet-{pet_name}')
action_list = ['running the park', 'taking a break', 'playing with friends', 'eating dog food', 'wearing clothes', 'sleeping', 'at home', 'looking cute', 'barking']
len_action = len(action_list)


start loading...


text_encoder/model.safetensors not found
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [11]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import FileResponse
from fastapi.responses import StreamingResponse
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from io import BytesIO
import random

app = FastAPI()

# -----------------------------------------------------------------------

# pipe 에 이미 받아놓은 모델이 있는 상황,

@app.post('/generate')
async def generate_image():
    try:
        # Prompt 랜덤 샘플링
        idx = random.randint(0, len_action - 1 )
        action = action_list[idx]
        tuned_prompt = f'a photo of sks {pet_name} {action}'

        # Generate the image using the provided prompt
        image = generate(pipe, tuned_prompt)  # This should be your image object

        # Convert the image object to bytes
        img_io = BytesIO()
        image.save(img_io, 'JPEG')
        img_io.seek(0)

        # Return the image as a streaming response
        return StreamingResponse(img_io, media_type="image/jpeg")

    except Exception as e:
        # Handle exceptions and return a user-friendly message
        raise HTTPException(status_code=500, detail=str(e))

@app.get('/test')
def testAi():
    return {"msg":"hello world"}


# -----------------------------------------------------------------------

ngrok.set_auth_token("2bo2UxLR9yL2fQ43JKEm6USFs1u_7gsDKr6iuzemiTQFCbnLD")

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://c7f3-34-126-85-170.ngrok-free.app


INFO:     Started server process [468]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     27.0.238.117:0 - "GET / HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [468]
